# Lasso
Similar to the "RidgeRegression" class, we can implement the Lasso algorithm.

Please type in the codes in the specified space to complete the construction of the class ``Lasso``.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# THIS SHOULD BE ADDAPTED TO THE DATASETS

def simulation(m):
    """
    Generate a specified number of samples according to the sparse linear model.

    Parameters
    -----
    m : num_samples

    Returns
    -----
    x (matrix, m*num_variables) : Input or features 
    y (matrix, m*1): Output or labels
    """

    # Generate independent and identically distributed samples as inputs.
    x1 = np.random.normal(3,1,[m,1])
    x2 = np.random.uniform(0,1,[m,1])
    x3 = np.random.normal(1,4,[m,1])
    x4 = np.random.normal(-1,1,[m,1])
    x5 = np.random.normal(0,1,[m,1])
    x6 = np.random.uniform(-1,1,[m,1])
    # Generate the true outputs according to the sparse linear model.
    y = x1 + 3*x2 + 2 + np.random.normal(0,0.1,[m,1])
    return np.hstack([x1,x2,x3,x4,x5,x6]), y

# Generate 5000 samples and split them into training and test dataset.
X, y = simulation(5000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)

In [ ]:
class Lasso():
    '''
    This is a class for Lasso algorithm.
    
    The class contains the hyper parameters of the lasso algorithm as attributes, such as the regurization 
    parameter(Lambda) of L_1 penality.
    It also contains the functions for initializing the class, fitting the lasso model and use the fitted 
    model to predict test samples.
    
    Attributes:
        Lambda:    regularization parameter for L_1 penalty
        max_itr:   maximum number of iteration for gradient descent
        tol:       if the change in loss is smaller than tol, then we stop iteration
        W:         concatenation of weight w and bias b
        
    '''
    def __init__(self, Lambda=0.5, max_itr=100, tol=0.0001):
        '''
        Initialize the RidgeRegression class
        '''
        self.Lambda = Lambda
        self.max_itr = max_itr
        self.tol = tol  
    
    def _loss_lasso(self, X, y, W):
        '''
        Calculating the regularized empirical loss
        '''
        return ((y-X@W).T@(y-X@W))[0,0] + self.Lambda * np.sum(np.abs(W[:X.shape[1]-1,0]))
    
    def fit(self, x, y):
        '''
        estimate the weight and bias in the lasso model by coordinate gradient descent
        
        Args: 
            x (matrix, num_train*num_variables): input of training samples
            y (matrix, num_test*1): output of training samples
            
        Returns:
            self.W (matrix, (num_variables+1)*1): estimation of weight w and bias b
        '''
        m = x.shape[0]
        ### Add the all-one vector to the last column 
        X = np.concatenate((x,np.ones((m,1))),axis=1)
        # weight and bias initialization
        d = X.shape[1]
        self.W = np.zeros((d,1))
        
        ### Use the cooridinate gradient descent to update W
        previous_loss = self._loss_lasso(X, y, self.W)
        for i in range(self.max_itr):
            ### Update bias
            self.W[-1,0] = np.mean(y.T-x@self.W[:-1,0])
            ### Update W_j, j=0,...,d-2
            for j in range(d-1):
                # Calculate r_j = Y - X@W, with W[j,0]=0 and other elements in W unchanged
                copy_W = self.W.copy()
                copy_W[j,0] = 0
                rj = y - X@copy_W
                # Calculate X[:,j]@r_j and X[:,j].T@X[:,j]
                aj = X[:,j].T@X[:,j]
                bj = 2 * X[:,j]@rj / m 
                if bj <= -self.Lambda:
                    self.W[j,0] = (bj + self.Lambda)/(2*aj)*m
                elif bj >= self.Lambda:
                    self.W[j,0] = (bj - self.Lambda)/(2*aj)*m
                else:
                    self.W[j,0] = 0
            current_loss = self._loss_lasso(X, y, self.W)
            if previous_loss - current_loss < self.tol:
                print(f'Converged after {i} iterations')
                break
            else:
                previous_loss = current_loss
        return self.W
    
    def predict(self, x):
        '''
        predict the output of the test samples
        
        Args: 
            x (matrix, num_test*num_variables): input of test samples
            
        Returns:
            y (matrix, num_test*1): predicted outputs of test samples
        ''' 
        m = x.shape[0]
        X = np.concatenate((x,np.ones((m,1))),axis=1)
        return  X@self.W
from sklearn.metrics import mean_squared_error
### Initial the class Lasso by assigning values to the parameters.
model = Lasso(Lambda = 0.015, max_itr=10000, tol=1e-5)
### Fit model with training data
W = model.fit(X_train, y_train)
### Predict the output of test samples
y_pred = model.predict(X_test)
### Evaluate the model by calculating the MSE of test samples.
mse = mean_squared_error(y_pred, y_test)
### Print MSE 
print("MSE of Lasso is {}".format(mse))
### Print the estimated w and b
print("The weight w of Lasso is \n {}.".format(W[:X_test.shape[1],0].T))
print("The bias b of Lasso is {}.".format(W[X_test.shape[1],0]))